# Purpose

Weighter is designed to be a hacky weight tracking app using Slack as a frontend and Google Sheets as a database! 
Weights are entered through a Slack Channel, stored in a Google Sheet, and reported back to users through Slack. Users will have the option to view various stats and graphs by sending different slack messages. 

Weighter also features additive modeling forecasts using the Facebook Prophet library. 

## Setup Libraries and Access to the Google Sheet

In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# fbprophet for additive models
import fbprophet

# gspread for Google Sheets access
import gspread

# slacker for interacting with Slack
from slacker import Slacker

# oauth2client for authorizing access to Google Sheets
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# matplotlib for plotting in the notebook
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib

### Google Sheet Access

The json file is the credentials for accessing the google sheet generated from the Google Developers API. To access a specific sheet, you need to share the sheet with the email address in the json file. 

In [69]:
# google sheets access
scope = ['https://spreadsheets.google.com/feeds']

# Use local stored credentials in json file
# make sure to first share the sheet with the email in the json file
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/Will Koehrsen/Desktop/weighter-2038ffb4e5a6.json', scope)

# Authorize access
gc = gspread.authorize(credentials);

INFO:oauth2client.client:Refreshing access_token


## Set up Slack Access

In [70]:
# Slack api key is stored as text file
with open('C:/Users/Will Koehrsen/Desktop/slack_api.txt', 'r') as f:
    slack_api_key = f.read()

In [72]:
slack = Slacker(slack_api_key)

In [16]:
slack.chat.post_message('#test_python', 'Hello Fellow Slackers')

### Open the sheet and convert to a pandas dataframe

In [4]:
# Open the sheet, need to share the sheet with email specified in json file
gsheet = gc.open('Auto Weight Challenge').sheet1

# List of lists with each row in the sheet as a list
weight_lists = gsheet.get_all_values()

# Headers are the first list
# Pop returns the element (list in this case) and removes it from the list
headers = weight_lists.pop(0)

# Convert list of lists to a dataframe with specified column header
weights = pd.DataFrame(weight_lists, columns=headers)

# Record column should be a boolean
weights['Record'] = weights['Record'].astype(bool)

# Name column is a string
weights['Name'] = weights['Name'].astype(str)

# Convert dates to datetime, then set as index, then set the time zone
weights['Date'] = pd.to_datetime(weights['Date'], unit='s')
weights  = weights.set_index('Date', drop = True).tz_localize(tz='US/Eastern')

In [5]:
weights.head()

,Name,Entry,Record
Date,,,
2017-08-18 00:00:00-04:00,koehrcl,235.2,True
2017-08-19 00:00:00-04:00,koehrcl,235.6,True
2017-08-20 00:00:00-04:00,koehrcl,233,True
2017-08-21 00:00:00-04:00,koehrcl,232.6,True
2017-08-22 00:00:00-04:00,koehrcl,234.4,True


In [6]:
weights.tail()

,Name,Entry,Record
Date,,,
2018-01-19 14:34:56-05:00,koehrcl,221.1,False
2018-01-19 15:30:25-05:00,willkoehrsen,137.3,False
2018-01-19 18:11:49-05:00,fletcher,188.4,False
2018-01-20 15:39:12-05:00,willkoehrsen,137,False
2018-01-20 15:49:52-05:00,koehrcl,220.4,False


+ Date is the index (in Eastern time here)
+ Name is the slack username
+ Entry is either weight or a string to display results
+ Record is whether or not the entry has been processed by weighter

# Weighter Class

The class will include a number of different methods for analyzing the data and graphing results. These results can then be sent back to Slack depending on the message entered by the user.

In [66]:
class Weighter():
    
    """
    When weighter is initialized, we need to convert the usernames,
    get a dictionary of the unrecorded entries, construct a dictionary
    of the actions to take, and make sure all data is formatted correctly
    """
    
    def __init__(self, weights):
        
        # Weights is a dataframe
        self.weights = weights.copy()
        
        # Users is a list of the unique users in the data
        self.users = list(set(self.weights['Name']))
        
        correct_names = []
        # Name changes
        for user in self.weights['Name']:
            
            # Have to hardcode in name changes
            if user == 'koehrcl':
                correct_names.append('Craig')
            elif user == 'willkoehrsen':
                correct_names.append('Will')
            elif user == 'fletcher':
                correct_names.append('Fletcher')
            
            # Currently do not handle new users
            else:
                print('New User Detected')
                return
            
        self.weights['Name'] = correct_names
        
        # Users is a list of the unique users in the data
        self.users = list(set(self.weights['Name']))
        
        # Create a dataframe of the unrecorded entries
        self.unrecorded = self.weights[self.weights['Record'] != True]
        
        # Process the unrecorded entries
        self.process_unrecorded()
        
        # The remaning entries will all be weights
        self.weights['Entry'] = [float(weight) for weight in self.weights['Entry']]
        
        # Build the user dictionary
        self.build_user_dict()
        
        
    """ 
    Constructs a dictionary for each user with critical information
    This forms the basis for the summarize function
    """
    
    def build_user_dict(self):
        
        user_dict = {}
        
        user_goals = {'Craig': 215.0, 'Fletcher': 200.0, 'Will': 155.0}
        
        for i, user in enumerate(self.users):
            
            user_weights = self.weights[self.weights['Name'] == user]
            goal = user_goals.get(user)

            start_weight = user_weights.ix[min(user_weights.index), 'Entry']   
            start_date = min(user_weights.index)
            
            # Find minimum weight and date on which it occurs
            min_weight =  min(user_weights['Entry'])
            min_weight_date = ((user_weights[user_weights['Entry'] == min_weight].index)[0])
            
            # Find maximum weight and date on which it occurs
            max_weight = max(user_weights['Entry'])
            max_weight_date = ((user_weights[user_weights['Entry'] == max_weight].index)[0])
            
            most_recent_weight = user_weights.ix[max(user_weights.index), 'Entry']
            
            if goal < start_weight:
                change = start_weight - most_recent_weight
                obj = 'lose'
            elif goal > start_weight:
                change = most_recent_weight - start_weight
                obj = 'gain'
                
            pct_change = 100 *change / start_weight
            
            pct_to_goal = 100 * (change / abs(start_weight - goal) )
            
            user_dict[user] = {'min_weight': min_weight, 'max_weight': max_weight,
                               'min_date': min_weight_date, 'max_date': max_weight_date,
                               'recent': most_recent_weight, 'abs_change': change,
                               'pct_change': pct_change, 'pct_towards_goal': pct_to_goal,
                               'start_weight': start_weight, 'start_date': start_date,
                               'goal_weight': goal, 'objective': obj}
       
        self.user_dict = user_dict
             
    """
    Builds a dictionary of unrecorded entries where each key is the user
    and the value is a list of weights and methods called for by the user.
    This dictionary is saved as the entries attribute of the class.
    """
    
    def process_unrecorded(self):
        
        entries = {name:[] for name in self.users}
        drop = []
        
        for index in self.unrecorded.index:

            entry = self.unrecorded.ix[index, 'Entry']
            user = str(self.unrecorded.ix[index, 'Name'])
            
            # Try and except does not seem like the best way to handle this
            try:
                entry = float(entry)
                entries[user].append(entry)
                
            except:  
                entry = str(entry)
                entries[user].append(entry)
                
                drop.append(index)
                
            self.weights.ix[index, 'Record'] = True
        # Drop the rows which do not contain a weight
        self.weights.drop(drop, axis=0, inplace=True)
        
        # Entries is all of the new entries
        self.entries = entries
        
    """ 
    Iterates through the unrecorded entries and delegates 
    each one to the appropriate method.
    """
    def process_entries(self):
        for user, user_entries in self.entries.items():
            for entry in user_entries:
                if type(entry) == float:
                    self.basic_message(user, entry)
                
                elif entry.lower() == 'summary':
                    self.summary(user)
                
    """ 
    This method is automatically run for each new weight
    """
    def basic_message(self, user, entry):
    
        # Find information for user, construct message, post message to Slack
        user_info = self.user_dict.get(user)

        message = ("\n{}: Total Weight Change = {:.2f} lbs.\n\n"
                    "Percentage Weight Change = {:.2f}%").format(user, user_info['abs_change'],
                                                     user_info['pct_change'])

        slack.chat.post_message('#test_python', text=message, username='Weight Challenge Update')
                        
    """ 
    Displays comprehensive stats about the user
    Only run on a summary message in the slack channel
    """
    
    def summary(self, user):
        user_info = self.user_dict.get(user)
        message = ("\n{}, your most recent weight was {:.2f} lbs.\n\n"
                   "Absolute weight change = {:.2f} lbs, percentage weight change = {:.2f}%.\n\n"
                   "Minimum weight = {:.2f} lbs on {} and maximum weight = {:.2f} lbs on {}.\n\n"
                   "Your goal weight = {:.2f} lbs. and you are {:.2f}% of the way there.\n\n"
                   "You started at {:.2f} lbs on {}. Congratulations on the progress!").format(user, 
                     user_info['recent'], user_info['abs_change'], user_info['pct_change'], 
                     user_info['min_weight'], str(user_info['min_date'].date()),
                     user_info['max_weight'], str(user_info['max_date'].date()),
                     user_info['goal_weight'], user_info['pct_towards_goal'],                                                       
                     user_info['start_weight'], str(user_info['start_date'].date()))
        
        slack.chat.post_message('#test_python', text=message, username='%s Summary' % user)
        
    

In [67]:
weighter = Weighter(weights)

In [68]:
weighter.weights.tail()

,Name,Entry,Record
Date,,,
2018-01-19 14:34:56-05:00,Craig,221.1,True
2018-01-19 15:30:25-05:00,Will,137.3,True
2018-01-19 18:11:49-05:00,Fletcher,188.4,True
2018-01-20 15:39:12-05:00,Will,137.0,True
2018-01-20 15:49:52-05:00,Craig,220.4,True


In [65]:
weighter.process_entries()

In [47]:
weighter.summary('Craig')


Craig, your most recent weight was 220.40 lbs.

Absolute weight change = 14.80 lbs and percentage weight change = 6.29%.

Your goal weight = 215.00 lbs. and you are 73.27% of the way there.

Minimum weight = 219.60 lbs on 2017-12-27 and maximum weight = 235.60 lbs on 2017-08-19.

You started at 235.20 lbs on 2017-08-18. Congratulations on the progress!


In [9]:
weighter.basic_message()


User: Fletcher
Total Weight Change = 4.00 lbs.
Percentage Weight Change = %2.17.

User: Fletcher
Total Weight Change = 3.80 lbs.
Percentage Weight Change = %2.06.

User: Craig
Total Weight Change = 15.40 lbs.
Percentage Weight Change = %6.54.

User: Craig
Total Weight Change = 14.50 lbs.
Percentage Weight Change = %6.15.

User: Craig
Total Weight Change = 15.20 lbs.
Percentage Weight Change = %6.45.

User: Will
Total Weight Change = 14.20 lbs.
Percentage Weight Change = %11.49.

User: Will
Total Weight Change = 13.70 lbs.
Percentage Weight Change = %11.08.

User: Will
Total Weight Change = 13.40 lbs.
Percentage Weight Change = %10.84.


In [10]:
weighter.summary('Craig')

{'min_weight': 219.6, 'max_weight': 235.6, 'min_date': Timestamp('2017-12-27 15:14:17-0500', tz='US/Eastern'), 'max_date': Timestamp('2017-08-19 00:00:00-0400', tz='US/Eastern'), 'recent': 220.40000000000001, 'abs_change': 14.799999999999983, 'pct_change': 6.2925170068027141, 'pct_towards_goal': 73.267326732673226, 'objective': 'lose'}
